In [28]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold, StratifiedKFold,\
GroupKFold, cross_validate, GridSearchCV
from sklearn.linear_model import LogisticRegression
from collections import Counter
%matplotlib inline

### 표본추출

#### 단순랜덤추출법

In [2]:
data = load_iris()
iris_cols = list(data['feature_names']) + ['target']
iris = pd.DataFrame(np.c_[data['data'], data['target']], columns=
                    [col.replace(' (cm)', '') for col in iris_cols])
print(iris.head(3))

   sepal length  sepal width  petal length  petal width  target
0           5.1          3.5           1.4          0.2     0.0
1           4.9          3.0           1.4          0.2     0.0
2           4.7          3.2           1.3          0.2     0.0


In [3]:
data_list = [1,2,3,4,5,'a','b','c']
random.sample(data_list, 4)

['a', 1, 2, 4]

In [4]:
# 넘파이 라이브러리
np.random.choice(data_list, 4, replace=True)

array(['5', 'c', '3', '1'], dtype='<U11')

In [5]:
# 특정 범위 내의 정수/실수의 난수 생성
print('0~10 사이의 정수 중 3개의 난수 생성: ', np.random.randint(0, 10, 3))
print('0~1 사이의 실수를 2*2 배열로 생성:\n', np.random.rand(2,2))

0~10 사이의 정수 중 3개의 난수 생성:  [9 2 5]
0~1 사이의 실수를 2*2 배열로 생성:
 [[0.57980441 0.22903363]
 [0.3809708  0.29916238]]


In [6]:
print(iris.sample(n=3, replace=False))

    sepal length  sepal width  petal length  petal width  target
6            4.6          3.4           1.4          0.3     0.0
54           6.5          2.8           4.6          1.5     1.0
9            4.9          3.1           1.5          0.1     0.0


In [7]:
# frac 파라미터로 전체 데이터의 3%를 랜덤으로 추출할 수 있음.
print(iris.sample(frac=0.03))

     sepal length  sepal width  petal length  petal width  target
42            4.4          3.2           1.3          0.2     0.0
72            6.3          2.5           4.9          1.5     1.0
106           4.9          2.5           4.5          1.7     2.0
35            5.0          3.2           1.2          0.2     0.0


In [8]:
print(iris.sample(frac=0.03, weights='sepal length'))

     sepal length  sepal width  petal length  petal width  target
134           6.1          2.6           5.6          1.4     2.0
120           6.9          3.2           5.7          2.3     2.0
63            6.1          2.9           4.7          1.4     1.0
17            5.1          3.5           1.4          0.3     0.0


In [9]:
print(iris.sample(3, axis=1).head(3))

   sepal width  sepal length  petal length
0          3.5           5.1           1.4
1          3.0           4.9           1.4
2          3.2           4.7           1.3


- 계통추출법

In [10]:
def sys_sampling(data, n) : 
    N = len(data)
    K = N//n
    index = data[:K].sample(1).index  # 첫 구간에서 임의로 선택한 샘플 1개의 인덱스
    # index개씩 띄어서 각 구간에서 하나씩 샘플 추출
    sys_df = pd.DataFrame()
    while len(sys_df) < n : 
        # sys_df = sys_df.append(data.loc[index,:])
        sys_df = pd.concat([sys_df, data.loc[index,:]])
        index += K
    return sys_df

print(sys_sampling(iris, 8))

     sepal length  sepal width  petal length  petal width  target
17            5.1          3.5           1.4          0.3     0.0
35            5.0          3.2           1.2          0.2     0.0
53            5.5          2.3           4.0          1.3     1.0
71            6.1          2.8           4.0          1.3     1.0
89            5.5          2.5           4.0          1.3     1.0
107           7.3          2.9           6.3          1.8     2.0
125           7.2          3.2           6.0          1.8     2.0
143           6.8          3.2           5.9          2.3     2.0


- 집락 추출법/층화추출법

In [11]:
def start_random_sampling(data, stratum, sampling_no, proportion=True) :
    if proportion == True : # 비례층화추출법: 원본 데이터 개수의 비율대로 추출
        levels = data[stratum].unique()
        total = data[stratum].value_counts().sum()
        prop_val = data[stratum].value_counts()/total
        no = prop_val * sampling_no
        result = pd.DataFrame()
        for level in levels : 
            temp_df = data[data[stratum]==level].sample(int(no[level]))
            result = pd.concat([result, temp_df])

    else :  # 불비례층화추출법: 임의로 정한 특정 비율대로 샘플링
        levels = list(proportion.keys())
        prop_val = np.array(list(proportion.values()))
        total = sum(prop_val)
        no = prop_val * sampling_no
        if total != 1 : 
            raise Exception('proportion sum is supposed to be 1.')
        else : 
            result = pd.DataFrame()
            for level in levels : 
                temp_df = data[data[stratum]==level].sample(int(no[level]))
                result = pd.concat([result, temp_df])

    return result

In [12]:
# 원본 층별 데이터 개수 확인: 층별로 동일하게 50개씩 관측값을 가지는 데이터
iris['target'].value_counts()

0.0    50
1.0    50
2.0    50
Name: target, dtype: int64

In [13]:
# 비례층화추출법으로 9개 샘플링하기: 원본과 동일한 비율로 샘플링하기
print(start_random_sampling(iris, 'target', 9))

     sepal length  sepal width  petal length  petal width  target
22            4.6          3.6           1.0          0.2     0.0
11            4.8          3.4           1.6          0.2     0.0
34            4.9          3.1           1.5          0.2     0.0
81            5.5          2.4           3.7          1.0     1.0
57            4.9          2.4           3.3          1.0     1.0
95            5.7          3.0           4.2          1.2     1.0
110           6.5          3.2           5.1          2.0     2.0
120           6.9          3.2           5.7          2.3     2.0
132           6.4          2.8           5.6          2.2     2.0


In [14]:
# 불비례층화추출법으로 10개 샘플링하기: 임의로 정한 비율로 샘플링하기
print(start_random_sampling(iris, 'target', 10, proportion={0:0.2, 1:0.5,
                                                            2:0.3}))

     sepal length  sepal width  petal length  petal width  target
26            5.0          3.4           1.6          0.4     0.0
17            5.1          3.5           1.4          0.3     0.0
80            5.5          2.4           3.8          1.1     1.0
84            5.4          3.0           4.5          1.5     1.0
85            6.0          3.4           4.5          1.6     1.0
81            5.5          2.4           3.7          1.0     1.0
93            5.0          2.3           3.3          1.0     1.0
148           6.2          3.4           5.4          2.3     2.0
147           6.5          3.0           5.2          2.0     2.0
133           6.3          2.8           5.1          1.5     2.0


### 데이터 분할

- 일반적인 데이터 분할 및 홀드아웃 방법

In [15]:
X = iris.drop('target', axis=1)
y = iris.filter(['target'])

# 일반적 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print('X_train(y_train): %d(%d), X_test(y_test): %d(%d)' % (len(X_train), len(y_train), len(X_test), len(y_test)))
print('X_train의 비율: %0.2f, X_test의 비율: %0.2f' % (len(X_train)/len(X), len(X_test)/len(X)), '\n')

# 홀드아웃 방법
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
print('X_train(y_train): %d(%d), X_test(y_test) %d(%d)' % (len(X_train), len(y_train), len(X_test), len(y_test)))
print('X_train의 비율: %0.2f, X_test의 비율: %0.2f' % (len(X_train)/len(X), len(X_test)/len(X)))

X_train(y_train): 105(105), X_test(y_test): 45(45)
X_train의 비율: 0.70, X_test의 비율: 0.30 

X_train(y_train): 75(75), X_test(y_test) 75(75)
X_train의 비율: 0.50, X_test의 비율: 0.50


- Bootstrap: 랜덤복원추출

In [16]:
def Bootstrap(X, y, test_size = 0.3) : 
    train_size = 1 - test_size
    train_X = X.sample(frac=train_size)
    sampled_train_id = train_X.index
    sampled_test_id = X.drop(train_X.index, axis=0).index
    train_y = y.iloc[sampled_train_id,:]
    test_X = X.iloc[sampled_test_id,:]
    test_y = y.iloc[sampled_test_id,:]
    return train_X, test_X, train_y, test_y

for i in range(3) : 
    X_train, X_test, y_train, y_test = Bootstrap(X, y)
    print(f'Sample {i} ==> train_index: {X_train.index[:3]}, test_index: {X_test.index[:3]}')
    print('\tX_train의 비율: %0.2f, X_test의 비율: %0.2f' % (len(X_train)/len(X), len(X_test)/len(X)))
    print('\ty_train의 타겟 구성:', Counter(y_train['target']))
    print('\ty_test의 타겟 구성:', Counter(y_test['target']), '\n')


Sample 0 ==> train_index: Int64Index([17, 24, 9], dtype='int64'), test_index: Int64Index([0, 2, 3], dtype='int64')
	X_train의 비율: 0.70, X_test의 비율: 0.30
	y_train의 타겟 구성: Counter({1.0: 38, 0.0: 34, 2.0: 33})
	y_test의 타겟 구성: Counter({2.0: 17, 0.0: 16, 1.0: 12}) 

Sample 1 ==> train_index: Int64Index([49, 13, 79], dtype='int64'), test_index: Int64Index([1, 8, 9], dtype='int64')
	X_train의 비율: 0.70, X_test의 비율: 0.30
	y_train의 타겟 구성: Counter({1.0: 38, 2.0: 35, 0.0: 32})
	y_test의 타겟 구성: Counter({0.0: 18, 2.0: 15, 1.0: 12}) 

Sample 2 ==> train_index: Int64Index([18, 48, 28], dtype='int64'), test_index: Int64Index([1, 2, 8], dtype='int64')
	X_train의 비율: 0.70, X_test의 비율: 0.30
	y_train의 타겟 구성: Counter({0.0: 36, 1.0: 36, 2.0: 33})
	y_test의 타겟 구성: Counter({2.0: 17, 0.0: 14, 1.0: 14}) 



- Shuffle Split: 무작위 순열 교차 검증

In [17]:
ss = ShuffleSplit(test_size=0.5, train_size=0.5, n_splits=3)
for i, (train_index, test_index) in enumerate(ss.split(X)) : 
    print(f'Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}')
    X_train, X_test, y_train, y_test = X.iloc[train_index,:], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print('\tX_train의 비율: %0.2f, X_test의 비율: %0.2f' % (len(X_train)/len(X), len(X_test)/len(X)))
    print('\ty_train의 타겟 구성:', Counter(y_train['target']))
    print('\ty_test의 타겟 구성:', Counter(y_test['target']), '\n')

Sample 0 ==> train_index: [ 19  31 117], test_index: [  3  59 147]
	X_train의 비율: 0.50, X_test의 비율: 0.50
	y_train의 타겟 구성: Counter({1.0: 30, 2.0: 24, 0.0: 21})
	y_test의 타겟 구성: Counter({0.0: 29, 2.0: 26, 1.0: 20}) 

Sample 1 ==> train_index: [45 92  7], test_index: [116  84  12]
	X_train의 비율: 0.50, X_test의 비율: 0.50
	y_train의 타겟 구성: Counter({1.0: 26, 2.0: 26, 0.0: 23})
	y_test의 타겟 구성: Counter({0.0: 27, 2.0: 24, 1.0: 24}) 

Sample 2 ==> train_index: [ 72  49 144], test_index: [107  44 122]
	X_train의 비율: 0.50, X_test의 비율: 0.50
	y_train의 타겟 구성: Counter({1.0: 26, 0.0: 25, 2.0: 24})
	y_test의 타겟 구성: Counter({2.0: 26, 0.0: 25, 1.0: 24}) 



- K-fold 분할

In [18]:
kf = KFold(n_splits=4, shuffle=False)
for i, (train_index, test_index) in enumerate(kf.split(X)) : 
    print(f'Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}')
    X_train, X_test, y_train, y_test = X.iloc[train_index,:], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print('\tX_train의 비율: %0.2f, X_test의 비율: %0.2f' % (len(X_train)/len(X), len(X_test)/len(X)))
    print('\ty_train의 타겟 구성:', Counter(y_train['target']))
    print('\ty_test의 타겟 구성:', Counter(y_test['target']), '\n')

Sample 0 ==> train_index: [38 39 40], test_index: [0 1 2]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({1.0: 50, 2.0: 50, 0.0: 12})
	y_test의 타겟 구성: Counter({0.0: 38}) 

Sample 1 ==> train_index: [0 1 2], test_index: [38 39 40]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({2.0: 50, 0.0: 38, 1.0: 24})
	y_test의 타겟 구성: Counter({1.0: 26, 0.0: 12}) 

Sample 2 ==> train_index: [0 1 2], test_index: [76 77 78]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 50, 2.0: 37, 1.0: 26})
	y_test의 타겟 구성: Counter({1.0: 24, 2.0: 13}) 

Sample 3 ==> train_index: [0 1 2], test_index: [113 114 115]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 50, 1.0: 50, 2.0: 13})
	y_test의 타겟 구성: Counter({2.0: 37}) 



- Stratified K-fold 분할: 불균형 방지

In [19]:
skf = StratifiedKFold(n_splits=4)
# 분할 시 y를 고려해야 하기 때문에 split에 y를 입력해 주어야 함.
for i, (train_index, test_index) in enumerate(skf.split(X, y)) : 
    print(f'Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}')
    X_train, X_test, y_train, y_test = X.iloc[train_index,:], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print('\tX_train의 비율: %0.2f, X_test의 비율: %0.2f' % (len(X_train)/len(X), len(X_test)/len(X)))
    print('\ty_train의 타겟 구성:', Counter(y_train['target']))
    print('\ty_test의 타겟 구성:', Counter(y_test['target']))

Sample 0 ==> train_index: [13 14 15], test_index: [0 1 2]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({1.0: 38, 0.0: 37, 2.0: 37})
	y_test의 타겟 구성: Counter({0.0: 13, 2.0: 13, 1.0: 12})
Sample 1 ==> train_index: [0 1 2], test_index: [13 14 15]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({1.0: 38, 0.0: 37, 2.0: 37})
	y_test의 타겟 구성: Counter({0.0: 13, 2.0: 13, 1.0: 12})
Sample 2 ==> train_index: [0 1 2], test_index: [26 27 28]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 38, 2.0: 38, 1.0: 37})
	y_test의 타겟 구성: Counter({1.0: 13, 0.0: 12, 2.0: 12})
Sample 3 ==> train_index: [0 1 2], test_index: [38 39 40]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 38, 2.0: 38, 1.0: 37})
	y_test의 타겟 구성: Counter({1.0: 13, 0.0: 12, 2.0: 12})


- Group K-fold 분할

In [20]:
# group이 있는 데이터 생성하기
# group은 0,1,2,3의 4종류가 있음.
iris2 = iris.copy()
iris2['group'] = iris2['target'].apply(lambda x: int(np.random.randint(0,4,1)))
print(iris2.head(3))

   sepal length  sepal width  petal length  petal width  target  group
0           5.1          3.5           1.4          0.2     0.0      3
1           4.9          3.0           1.4          0.2     0.0      2
2           4.7          3.2           1.3          0.2     0.0      3


In [23]:
X = iris2.drop(['target', 'group'], axis=1)
y = iris2.filter(['target'])
group = iris2.filter(['group'])
gkf = GroupKFold(n_splits=4)
for i, (train_index, test_index) in enumerate(gkf.split(X, y, group)) : 
    print(f'Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}')
    X_train, X_test, y_train, y_test = X.iloc[train_index,:], X.iloc[test_index], \
        y.iloc[train_index], y.iloc[test_index]
    print('\tX_train의 비율: %0.2f, X_test의 비율: %0.2f' % (len(X_train)/len(X), len(X_test)/len(X)))
    print('\ttrain의 타겟 구성:', Counter(y_train['target']))
    print('\ttest의 타겟 구성:', Counter(y_test['target']))
    print('\ttrain의 그룹 구성:', Counter(group.iloc[train_index]['group']))
    print('\ttest의 그룹 구성:', Counter(group.iloc[test_index]['group']))

Sample 0 ==> train_index: [1 3 4], test_index: [0 2 7]
	X_train의 비율: 0.69, X_test의 비율: 0.31
	train의 타겟 구성: Counter({1.0: 37, 2.0: 36, 0.0: 31})
	test의 타겟 구성: Counter({0.0: 19, 2.0: 14, 1.0: 13})
	train의 그룹 구성: Counter({0: 38, 2: 36, 1: 30})
	test의 그룹 구성: Counter({3: 46})
Sample 1 ==> train_index: [0 1 2], test_index: [4 5 6]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	train의 타겟 구성: Counter({1.0: 40, 0.0: 37, 2.0: 35})
	test의 타겟 구성: Counter({2.0: 15, 0.0: 13, 1.0: 10})
	train의 그룹 구성: Counter({3: 46, 2: 36, 1: 30})
	test의 그룹 구성: Counter({0: 38})
Sample 2 ==> train_index: [0 2 4], test_index: [1 3 9]
	X_train의 비율: 0.76, X_test의 비율: 0.24
	train의 타겟 구성: Counter({2.0: 41, 0.0: 40, 1.0: 33})
	test의 타겟 구성: Counter({1.0: 17, 0.0: 10, 2.0: 9})
	train의 그룹 구성: Counter({3: 46, 0: 38, 1: 30})
	test의 그룹 구성: Counter({2: 36})
Sample 3 ==> train_index: [0 1 2], test_index: [15 17 19]
	X_train의 비율: 0.80, X_test의 비율: 0.20
	train의 타겟 구성: Counter({0.0: 42, 1.0: 40, 2.0: 38})
	test의 타겟 구성: Counter({2.0: 12, 1.0: 1

### 교차 검증

- 분할 샘플들로 교차 검증

In [26]:
X = iris.drop('target', axis=1)
y = iris['target']

LOGREG = LogisticRegression(max_iter=300, C=0.1)

# 샘플 분할 방법 정의하기
SKF = StratifiedKFold(n_splits=4)

# 교차 검증 실시하기
result = cross_validate(LOGREG, X, y, cv = SKF, return_train_score=True)
print(pd.DataFrame(result))

   fit_time  score_time  test_score  train_score
0  0.072053    0.002004    0.894737     0.955357
1  0.007999    0.001001    0.947368     0.964286
2  0.008000    0.001001    0.945946     0.955752
3  0.007999    0.001001    1.000000     0.938053


- 파라미터 후보들로 교차 검증

In [29]:
LOGREG = LogisticRegression(max_iter=300, C=0.1)

# 알고리즘의 파라미터 후보군을 정의하기
param_grid = {'C':[0.01, 0.1, 1], 'solver':['lbfgs', 'liblinear']}

SKF = StratifiedKFold(n_splits=2)

grid = GridSearchCV(LOGREG, param_grid, cv=SKF)
grid.fit(X,y)

print(f'최상의 교차 검증 점수: {grid.best_score_:.2f}')
print(f'최적의 매개변수: {grid.best_params_}')
print(pd.DataFrame(grid.cv_results_))

최상의 교차 검증 점수: 0.96
최적의 매개변수: {'C': 1, 'solver': 'lbfgs'}
   mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0       0.011512  4.523396e-03         0.001488        0.000521    0.01   
1       0.003503  4.951954e-04         0.000996        0.000004    0.01   
2       0.007499  5.006790e-04         0.000500        0.000500     0.1   
3       0.001500  4.999638e-04         0.001502        0.000502     0.1   
4       0.014081  8.511543e-05         0.001003        0.000002       1   
5       0.001007  9.536743e-07         0.001006        0.000000       1   

  param_solver                              params  split0_test_score  \
0        lbfgs      {'C': 0.01, 'solver': 'lbfgs'}           0.800000   
1    liblinear  {'C': 0.01, 'solver': 'liblinear'}           0.666667   
2        lbfgs       {'C': 0.1, 'solver': 'lbfgs'}           0.920000   
3    liblinear   {'C': 0.1, 'solver': 'liblinear'}           0.786667   
4        lbfgs         {'C': 1, 'solver': 'lbfgs'}  